# DEA Opioid Data

- Downloaded from here on July 30, 2021 as gzipped .tsv: https://d2ty8gaf6rmowa.cloudfront.net/dea-pain-pill-database/bulk/arcos_all_washpost.tsv.gz (clicking starts download)
- Extracted and uploaded to **coiled-datasets** S3 bucket


In [1]:
import coiled
import dask.dataframe as dd
from dask.distributed import Client

In [2]:
cluster = coiled.Cluster(
    name="data-import",
    software='rrpelgrim/coiled-base-m1',
    n_workers=20,
    worker_cpu=4,
    worker_memory='32Gb',
    scheduler_cpu=2,
    shutdown_on_close=False,
    scheduler_options={'idle_timeout':'1hour'},
)

Output()

Found software environment build
Created FW rules: coiled-dask-rrpelgr71-34294-firewall
Created scheduler VM: coiled-dask-rrpelgr71-34294-scheduler (ip: ['54.226.151.115'])


In [3]:
client = Client(cluster)
client

<Client: 'tls://10.4.0.158:8786' processes=11 threads=44, memory=319.00 GiB>

In [42]:
client.restart()

<Client: 'tls://10.4.0.73:8786' processes=0 threads=0, memory=0 B>

In [4]:
from coiled import performance_report
import distributed

with performance_report('read_table.html'):
    ddf = dd.read_table(
        "s3://coiled-datasets/dea-opioid/arcos_washpost.tsv",
        dtype={
            'ACTION_INDICATOR': 'object',
            'ORDER_FORM_NO': 'object',
            'REPORTER_ADDL_CO_INFO': 'object',
            'REPORTER_ADDRESS2': 'object',
            'UNIT': 'object',
            'NDC_NO': 'object',
        }
    )

    ddf = ddf.repartition(partition_size='100MB').persist()
    distributed.wait(ddf)

/Users/rpelgrim/mambaforge/envs/coiled-base/lib/python3.8/site-packages/coiled/utils.py:396: ExperimentalFeatureWarning: performance_report is an experimental feature which is subject to breaking changes, being removed, or otherwise updated without notice and should be used accordingly.
  warnings.warn(


]8;id=791575;https://cloud.coiled.io/rrpelgrim/reports/727\Performance Report Available at: https://cloud.coiled.io/rrpelgrim/reports/727]8;;\

In [5]:
ddf.head()

,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,...,DOSAGE_UNIT,TRANSACTION_ID,Product_Name,Ingredient_Name,Measure,MME_Conversion_Factor,Combined_Labeler_Name,Revised_Company_Name,Reporter_family,dos_str
0,PA0006836,DISTRIBUTOR,ACE SURGICAL SUPPLY CO INC,NaN,1034 PEARL STREET,NaN,BROCKTON,MA,2301,PLYMOUTH,...,100.0,64,HYDROCODONE BIT/ACETA 10MG/500MG USP,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,SpecGx LLC,Mallinckrodt,ACE Surgical Supply Co Inc,10.0
1,PA0021179,DISTRIBUTOR,APOTHECA INC,NaN,1622 N 16TH ST,NaN,PHOENIX,AZ,85006,MARICOPA,...,40.0,52,HYDROCODONE BITARTRATE & ACETA 5MG/,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Apotheca Inc.,Apotheca Inc.,Apotheca Inc,5.0
2,PA0021179,DISTRIBUTOR,APOTHECA INC,NaN,1622 N 16TH ST,NaN,PHOENIX,AZ,85006,MARICOPA,...,1200.0,119,HYDROCODONE BITARTRATE & ACETA 5MG/,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Apotheca Inc.,Apotheca Inc.,Apotheca Inc,5.0
3,PA0021179,DISTRIBUTOR,APOTHECA INC,NaN,1622 N 16TH ST,NaN,PHOENIX,AZ,85006,MARICOPA,...,600.0,34,HYDROCODONEBITARTRATE & ACETA 7.5MG,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Apotheca Inc.,Apotheca Inc.,Apotheca Inc,7.5
4,PA0021179,DISTRIBUTOR,APOTHECA INC,NaN,1622 N 16TH ST,NaN,PHOENIX,AZ,85006,MARICOPA,...,300.0,19,HYDROCODONE BITARTRATE & ACETA 5MG/,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Apotheca Inc.,Apotheca Inc.,Apotheca Inc,5.0


In [6]:
ddf.groupby('REPORTER_CITY').DOSAGE_UNIT.mean().compute()

REPORTER_CITY
BROCKTON              300.000000
COLUMBUS              237.618326
GOUVERNEUR            655.202560
GRANDVIEW HEIGHTS     797.290070
GUAYNABO              500.919179
                        ...     
EAST LIVERPOOL       5600.000000
SUGAR GROVE           100.000000
NEWTOWN              2800.000000
LINCOLN                 4.000000
NEW HYDE PARK           5.000000
Name: DOSAGE_UNIT, Length: 418, dtype: float64

In [30]:
ddf.groupby('REPORTER_CITY').STRENGTH.mean().compute()

REPORTER_CITY
BALTIMORE            NaN
BROCKTON             NaN
BURR RIDGE           NaN
CAGUAS               NaN
COLUMBUS             0.0
                   ...  
EAST LIVERPOOL       NaN
SUGAR GROVE       1000.0
NEWTOWN              0.0
LINCOLN              0.0
NEW HYDE PARK      100.0
Name: STRENGTH, Length: 418, dtype: float64

In [28]:
ddf.dtypes

REPORTER_DEA_NO           object
REPORTER_BUS_ACT          object
REPORTER_NAME             object
REPORTER_ADDL_CO_INFO     object
REPORTER_ADDRESS1         object
REPORTER_ADDRESS2         object
REPORTER_CITY             object
REPORTER_STATE            object
REPORTER_ZIP               int64
REPORTER_COUNTY           object
BUYER_DEA_NO              object
BUYER_BUS_ACT             object
BUYER_NAME                object
BUYER_ADDL_CO_INFO        object
BUYER_ADDRESS1            object
BUYER_ADDRESS2            object
BUYER_CITY                object
BUYER_STATE               object
BUYER_ZIP                  int64
BUYER_COUNTY              object
TRANSACTION_CODE          object
DRUG_CODE                  int64
NDC_NO                    object
DRUG_NAME                 object
QUANTITY                 float64
UNIT                      object
ACTION_INDICATOR          object
ORDER_FORM_NO             object
CORRECTION_NO            float64
STRENGTH                 float64
TRANSACTIO

In [11]:
ddf.columns

Index(['REPORTER_DEA_NO', 'REPORTER_BUS_ACT', 'REPORTER_NAME',
       'REPORTER_ADDL_CO_INFO', 'REPORTER_ADDRESS1', 'REPORTER_ADDRESS2',
       'REPORTER_CITY', 'REPORTER_STATE', 'REPORTER_ZIP', 'REPORTER_COUNTY',
       'BUYER_DEA_NO', 'BUYER_BUS_ACT', 'BUYER_NAME', 'BUYER_ADDL_CO_INFO',
       'BUYER_ADDRESS1', 'BUYER_ADDRESS2', 'BUYER_CITY', 'BUYER_STATE',
       'BUYER_ZIP', 'BUYER_COUNTY', 'TRANSACTION_CODE', 'DRUG_CODE', 'NDC_NO',
       'DRUG_NAME', 'QUANTITY', 'UNIT', 'ACTION_INDICATOR', 'ORDER_FORM_NO',
       'CORRECTION_NO', 'STRENGTH', 'TRANSACTION_DATE', 'CALC_BASE_WT_IN_GM',
       'DOSAGE_UNIT', 'TRANSACTION_ID', 'Product_Name', 'Ingredient_Name',
       'Measure', 'MME_Conversion_Factor', 'Combined_Labeler_Name',
       'Revised_Company_Name', 'Reporter_family', 'dos_str'],
      dtype='object')

In [32]:
ddf.REPORTER_DEA_NO.sample(frac=0.001).compute()

26603     PC0003044
116620    PF0000012
116567    PF0000012
59913     PD0029567
39201     PC0003044
            ...    
112109    RA0289050
133008    RA0289062
118820    RA0289050
90247     RA0289000
63391     RA0287020
Name: REPORTER_DEA_NO, Length: 178596, dtype: object

In [ ]:
ddf.to_parquet(
    )